In [344]:
#import the Basic libraries and packages
import pandas as pd
import numpy as np
import nltk
import string
import re

In [345]:
#Load the dataset and give a cloumn names
data = pd.read_csv("nlpdata.txt",sep=",,,",header=None)
data.columns = ['Question','Type']
data.head()

C:\Users\Devil\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,Question,Type
0,how did serfdom develop in and then leave russ...,unknown
1,what films featured the character popeye doyle ?,what
2,how can i find a list of celebrities ' real na...,unknown
3,what fowl grabs the spotlight after the chines...,what
4,what is the full form of .com ?,what


In [346]:
#split the columns question and answer
data['Type']=data['Type'].str.strip()
print(data['Type'].values)
data['Question'].values

['unknown' 'what' 'unknown' ... 'affirmation' 'affirmation' 'affirmation']


array(['how did serfdom develop in and then leave russia ? ',
       'what films featured the character popeye doyle ? ',
       "how can i find a list of celebrities ' real names ? ", ...,
       'does this hose have one ? ', 'can i get it in india ? ',
       'would this work on a 2008 ford edge with a naked roof ? '],
      dtype=object)

In [347]:
#covert the question into lower caces and apply regular expressions
data['Question'] = data['Question'].apply(lambda x: x.lower())
data['Question'] = data['Question'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [348]:
#remove the punctuvations in the dataset.
def count_Punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)-text.count(" ")),3)*100
    
data['Body-Lenth'] = data['Question'].apply(lambda x: len(x)- x.count(" "))
data['Punct %'] = data['Question'].apply(lambda x: count_Punct(x))
data.head()

,Question,Type,Body-Lenth,Punct %
0,how did serfdom develop in and then leave russ...,unknown,40,0.0
1,what films featured the character popeye doyle,what,40,0.0
2,how can i find a list of celebrities real nam...,unknown,38,0.0
3,what fowl grabs the spotlight after the chines...,what,55,0.0
4,what is the full form of com,what,22,0.0


In [349]:
#using TfidfVectorizer we generate vector matrix and also we used PorterStemmer alogorithem for stemming
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stopwords = nltk.corpus.stopwords.words('English')

def clean_text(text):
    ps = PorterStemmer()
    text = "".join([words.lower() for words in text if words not in string.punctuation])
    tokens = re.split("\W+",text)
    text = [ps.stem(words) for words in tokens if words not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
x_tfidf = tfidf_vect.fit_transform(data['Question'])
df = pd.DataFrame(x_tfidf.toarray())
df.head()

,0,1,2,3,4,5,6,7,8,9,...,3102,3103,3104,3105,3106,3107,3108,3109,3110,3111
0,0.072498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.067346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.072548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.056686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.082439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [350]:
#extract features 
x_feature = pd.concat([data['Body-Lenth'],data['Punct %'],pd.DataFrame(x_tfidf.toarray())],axis =1)
x_feature.head()


,Body-Lenth,Punct %,0,1,2,3,4,5,6,7,...,3102,3103,3104,3105,3106,3107,3108,3109,3110,3111
0,40,0.0,0.072498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40,0.0,0.067346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,38,0.0,0.072548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,55,0.0,0.056686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,22,0.0,0.082439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [351]:
#Alogrithmmn RF through CV
#When you don't bother much about interpreting the model but want better accuracy. 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold,cross_val_score
rf  = RandomForestClassifier(n_jobs=-1)
k_flod = KFold(n_splits = 5)
cross_val_score(rf,x_feature,data['Type'],cv=k_flod,scoring='accuracy',n_jobs=-1)

array([0.64983165, 0.62289562, 0.63299663, 0.39864865, 0.10810811])

In [352]:
#RF through Holdout set
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_feature,data['Type'],test_size=0.2)
rf  = RandomForestClassifier(n_estimators=50,max_depth=20,n_jobs=-1)
rf_model = rf.fit(x_train,y_train)
print('Accuracy:', rf.score(x_train,y_train))
#score(zip(rf_model.feature_importances_,x_train.columns),y_pred = rf_model.predict(x_test))
#precision,recall,fscore,support = score(y_test,y_predit,average='binary')
#print("precision{} / recall{} /accurecy{} ".format(round(precision),round(recall)))   

Accuracy: 0.5809443507588533


In [340]:
#TestQuestion=tfidf_vet.transform(["What time does the train leave"])
#rf_model.predict(TestQuestion)

In [367]:
#SKlearn's TF-IDF vectoriser transforms text data into sparse matrices. I can use these sparse matrices directly with
#a Naive Bayes classifier
tf_vectorizer = TfidfVectorizer(analyzer='word',lowercase=True)
X_train = tf_vectorizer.fit_transform(data.Question)

In [368]:
# split the data into a training set and a validation set
VALIDATION_SPLIT=0.20
indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)
X_train = X_train[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_train.shape[0])

x_train = X_train[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X_train[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [369]:
#naive Bayes classifiers creating simple yet well performing models,especially in the fields of text classification
#naive Bayes classifiers can outperform the more powerful alternatives.
from sklearn.naive_bayes import MultinomialNB
NBclafier = MultinomialNB()
NBclafier.fit(x_train,y_train)
NBclafier.predict(x_val)
print("Accuracy :",NBclafier.score(x_val,y_val))

Accuracy : 0.7567567567567568


In [370]:
#Finally test the model for the given input
TestQuestion=tfidf_vet.transform(["who markets spaghetti-o"])
NBclafier.predict(TestQuestion)

array([' who'], dtype='<U12')